In [ ]:
#!pip install sentence_transformers
#!pip install ag2[retrievechat]
#!pip install pyautogen==0.7.2
#https://microsoft.github.io/autogen/0.2/docs/notebooks/agentchat_RetrieveChat/

# Setting base for Retrive Chat

In [1]:
import json
import os

import chromadb

import autogen
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

# Accepted file formats for that can be stored in
# a vector database instance
from autogen.retrieve_utils import TEXT_FORMATS

In [2]:
print("Accepted file formats for `docs_path`:")
print(TEXT_FORMATS)

Accepted file formats for `docs_path`:
['txt', 'json', 'csv', 'tsv', 'md', 'html', 'htm', 'rtf', 'rst', 'jsonl', 'log', 'xml', 'yaml', 'yml', 'pdf', 'mdx']


In [12]:
config_list = [
    {
        # Let's choose the Meta's Llama 3.1 model (model names must match Ollama exactly)
        "model": "llama3.1",
        # We specify the API Type as 'ollama' so it uses the Ollama client class
        "api_type": "ollama",
        "stream": False,
        "client_host": "http://localhost:11434",
    }
]

In [ ]:
#import agentops
#agentops.init("954ecbcb-9555-470b-a2ad-a6999473648b")

In [13]:
# 1. create an AssistantAgent instance named "assistant"
assistant = AssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config={
        "timeout": 600,
        "cache_seed": 42,
        "config_list": config_list,
    },
)


In [14]:
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

# Extract text from local PDF files
pdf_files = ["ComprehensiveWarranty.pdf"]
#docs_content = [extract_text_from_pdf(file) for file in pdf_files]

from autogen.retrieve_utils import TEXT_FORMATS,extract_text_from_pdf
docs_content= extract_text_from_pdf("ComprehensiveWarranty.pdf")

In [15]:
# Define the RetrieveUserProxyAgent instance
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "code",
        "docs_path": ["autogen-docs/ComprehensiveWarranty.pdf"],
        "docs_content": docs_content,  # Provide the extracted content directly
        "chunk_token_size": 500,
        "model": "llama3.1",
        "vector_db": "chroma",
        "overwrite": True,
        "get_or_create": True,
    },
    code_execution_config=False,
)

# Process Starts for Scenario 1 : Generate code based off docstrings w/o human feedback

In [16]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

qa_problem = "Write a python code to check if warantee is covered or not. input will be year of purchase"
chat_result = ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem=qa_problem)
#agentops.end_session("Success")

Trying to create collection.


2025-03-22 16:44:11,797 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 3 chunks.
2025-03-22 16:44:11,825 - autogen.agentchat.contrib.vectordb.chromadb - INFO - No content embedding is provided. Will use the VectorDB's embedding function to generate the content embedding.
Number of requested results 20 is greater than number of elements in index 3, updating n_results = 3
Model llama3.1 not found. Using cl100k_base encoding.


VectorDB returns doc_ids:  [['470a433c', '59754bba', '6250035b']]
Adding content of doc 470a433c to context.


Model llama3.1 not found. Using cl100k_base encoding.


Adding content of doc 59754bba to context.


Model llama3.1 not found. Using cl100k_base encoding.


Adding content of doc 6250035b to context.
ragproxyagent (to assistant):

You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: Write a python code to check if warantee is covered or not. input will be year of purchase

Context is:  
• Unit must be purchased through authorized dealer of Godrej Appliances 
• Unit must preferably be installed by Godrej Appliances or its authorized service provider. 
• During the standard 1st year warranty period, the customer is entitled to 2 preventive 
maintenance services free of charge.  
• From 

Number of requested results 60 is greater than number of elements in index 3, updating n_results = 3


VectorDB returns doc_ids:  [['470a433c', '59754bba', '6250035b']]
No more context, will terminate.
ragproxyagent (to assistant):

TERMINATE

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (c85789b0-8586-4638-bea1-2c92d11fab3f): Termination message condition on agent 'assistant' met


In [ ]:
#agentops.end_session("Success")

# Process Starts for Scenario 2 : Answer a question based off docstrings w/o human feedback

In [17]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

ragproxyagent.task="qa"
qa_problem = "Is 5 year warantee applicable if I purchased the AC on January 2023"
chat_result = ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem=qa_problem)

Number of requested results 20 is greater than number of elements in index 3, updating n_results = 3
Model llama3.1 not found. Using cl100k_base encoding.


VectorDB returns doc_ids:  [['59754bba', '470a433c', '6250035b']]
Adding content of doc 59754bba to context.


Model llama3.1 not found. Using cl100k_base encoding.


Adding content of doc 470a433c to context.


Model llama3.1 not found. Using cl100k_base encoding.


Adding content of doc 6250035b to context.
ragproxyagent (to assistant):

You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: Is 5 year warantee applicable if I purchased the AC on January 2023

Context is:  
Internal 
 
5 YEAR COMPREHENSIVE AIR CONDITIONER WARRANTY OFFER 
 
Godrej & Boyce Mfg. Co. Ltd. (Godrej Appliances) hereby offers the following extended 
warranty to the original domestic purchaser of Godrej air conditioners commencing from the 
date of purchase of product by customer, applicable on installations within Indi

# Process Starts for Scenario 3 : Generate code based off docstrings w/ human feedback

In [18]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# set `human_input_mode` to be `ALWAYS`, so the agent will ask for human input at every step.
ragproxyagent.human_input_mode = "ALWAYS"
ragproxyagent.task="code"
code_problem = "Write a python code to check if warantee is covered or not. input will be year of purchase"
chat_result = ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem=code_problem)

Number of requested results 20 is greater than number of elements in index 3, updating n_results = 3
Model llama3.1 not found. Using cl100k_base encoding.


VectorDB returns doc_ids:  [['470a433c', '59754bba', '6250035b']]
Adding content of doc 470a433c to context.


Model llama3.1 not found. Using cl100k_base encoding.


Adding content of doc 59754bba to context.


Model llama3.1 not found. Using cl100k_base encoding.


...

ragproxyagent (to assistant):

I purchased the AC on 1st Jan 2025. What is the output of this program?

--------------------------------------------------------------------------------
assistant (to ragproxyagent):

Since you purchased the AC on January 1, 2025, which is before September 1, 2024, the output of the program will still be:

```
Warranty not covered
```

--------------------------------------------------------------------------------


Replying as ragproxyagent. Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  I purchased the AC on 1st Jan 2025. What is the output of this program?


ragproxyagent (to assistant):

I purchased the AC on 1st Jan 2025. What is the output of this program?

--------------------------------------------------------------------------------
assistant (to ragproxyagent):

Let's re-run the program with your input.

The purchase date is January 1, 2025, and the start date for the extended warranty offer is September 1, 2024. Since 2025 is greater than 2024, and any month after September is also valid, the output of the program will be:

```
Warranty is covered
```

--------------------------------------------------------------------------------


Replying as ragproxyagent. Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit



>>>>>>>> TERMINATING RUN (25ba2aa2-fe15-4950-8407-6e81904df50e): User requested to end the conversation


# Process Starts for Scenario 4: Answer a question based off docstrings w/ human feedback

In [19]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# set `human_input_mode` to be `ALWAYS`, so the agent will ask for human input at every step.
ragproxyagent.human_input_mode = "ALWAYS"
ragproxyagent.task="qa"
qa_problem = "Is 5 year warantee applicable if I purchased the AC on January 2023"
chat_result = ragproxyagent.initiate_chat(
    assistant, message=ragproxyagent.message_generator, problem=qa_problem
)  # type "exit" to exit the conversation

Number of requested results 20 is greater than number of elements in index 3, updating n_results = 3
Model llama3.1 not found. Using cl100k_base encoding.


VectorDB returns doc_ids:  [['59754bba', '470a433c', '6250035b']]
Adding content of doc 59754bba to context.


Model llama3.1 not found. Using cl100k_base encoding.


Adding content of doc 470a433c to context.


Model llama3.1 not found. Using cl100k_base encoding.


Adding content of doc 6250035b to context.
ragproxyagent (to assistant):

You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: Is 5 year warantee applicable if I purchased the AC on January 2023

Context is:  
Internal 
 
5 YEAR COMPREHENSIVE AIR CONDITIONER WARRANTY OFFER 
 
Godrej & Boyce Mfg. Co. Ltd. (Godrej Appliances) hereby offers the following extended 
warranty to the original domestic purchaser of Godrej air conditioners commencing from the 
date of purchase of product by customer, applicable on installations within Indi

Replying as ragproxyagent. Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  If I purchased the AC Januray 1st 2025. How long the warantee valid?


ragproxyagent (to assistant):

If I purchased the AC Januray 1st 2025. How long the warantee valid?

--------------------------------------------------------------------------------
assistant (to ragproxyagent):

The warranty offer is applicable from September 1st, 2024, onwards for individual and retail customers only.

Since you purchased the AC on January 1st, 2025, which is after September 1st, 2024, the extended warranty should be applicable to your purchase.

According to the offer, the air conditioner shall have a 5-year extended warranty (1 Year standard warranty + 4 Year extended warranty). Therefore, if you purchased the AC on January 1st, 2025, the warranty would be valid for 5 years from that date. 

So, the warranty would be valid until December 31st, 2030.

--------------------------------------------------------------------------------


Replying as ragproxyagent. Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit



>>>>>>>> TERMINATING RUN (8b206130-330c-4e16-9a6e-a320082b1c17): User requested to end the conversation
